# Lending Club Loan Prediction

In this notebook we will explore loan data from Lending Club between 2007 and 2019 to predict whether loan applicants will default on a loan or not.

The original data set can be found here: https://www.kaggle.com/wendykan/lending-club-loan-data

## Exploratory Data Analysis

We will begin by exploring this data set that was posted by Lending Club.

In [19]:
# Import libraries for initial data analysis

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Import sklearn modules
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

# Display plots in the notebook inline
%matplotlib inline

In [20]:
# Import our dataset
loans = pd.read_csv('./data/loan.csv', low_memory=False)

# View all columns
pd.set_option('display.max_columns', None)

#### Gather basic information

In [21]:
# A whopping 2.26M records with 145 features
loans.shape

(2260668, 145)

## Feature Engineering

The aim of this section is to create a model which can reliably predict the outcome of an issued loan based on the given features. This is a categorical model as the output will either be 1 denoting a good loan or 0 denoting a bad loan. In order to build a predictive model we will first have to do some feature engineering to prep our data set.

1.) Clean up features & records

2.) Eliminate class imbalance

3.) Deal with categorical and numeric features

### 1.) Clean up features & records

*Reduce number of features* - Out of the 145 features included with this data set, each requiring some cleanup. To prevent this from overwhelming, we'll just focus on a key set of features for now.

*Remove records prior to 2013* - Lending Club was still in growth mode between 2007-2015 (number of loans issued was doubling year over year) and their underwriting process was likely different than it is today. However we want to include loans which have reached full maturity (3 and 5 year terms) which will require us to include data as far back as 2013.

*Keep only Fully Paid and Charged off* - Remove records with loan_status other than either Fully Paid (good loan) or Charged Off (bad loan) to simplify prediction to a binary output. This will also include removing current loans since we have no way of predicting the outcome of current loan that has yet to come to term or be paid off.

*Remove joint applications* - There are a small number of loans which are joint applications. These loans should be treated differently than single applications as there are different features which will be more important when assessing each of these loan types.

In [22]:
# Let's take a look at some of the entries
loans.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,2500,2500,2500.0,36 months,13.56,84.92,C,C1,Chef,10+ years,RENT,55000.0,Not Verified,Dec-2018,Current,n,NaN,NaN,debt_consolidation,Debt consolidation,109xx,NY,18.24,0.0,Apr-2001,1.0,NaN,45.0,9.0,1.0,4341,10.3,34.0,w,2386.02,2386.02,167.02,167.02,113.98,53.04,0.0,0.0,0.0,Feb-2019,84.92,Mar-2019,Feb-2019,0.0,NaN,1,Individual,NaN,NaN,NaN,0.0,0.0,16901.0,2.0,2.0,1.0,2.0,2.0,12560.0,69.0,2.0,7.0,2137.0,28.0,42000.0,1.0,11.0,2.0,9.0,1878.0,34360.0,5.9,0.0,0.0,140.0,212.0,1.0,1.0,0.0,1.0,NaN,2.0,NaN,0.0,2.0,5.0,3.0,3.0,16.0,7.0,18.0,5.0,9.0,0.0,0.0,0.0,3.0,100.0,0.0,1.0,0.0,60124.0,16901.0,36500.0,18124.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,30000,30000,30000.0,60 months,18.94,777.23,D,D2,Postmaster,10+ years,MORTGAGE,90000.0,Source Verified,Dec-2018,Current,n,NaN,NaN,debt_consolidation,Debt consolidation,713xx,LA,26.52,0.0,Jun-1987,0.0,71.0,75.0,13.0,1.0,12315,24.2,44.0,w,29387.75,29387.75,1507.11,1507.11,612.25,894.86,0.0,0.0,0.0,Feb-2019,777.23,Mar-2019,Feb-2019,0.0,NaN,1,Individual,NaN,NaN,NaN,0.0,1208.0,321915.0,4.0,4.0,2.0,3.0,3.0,87153.0,88.0,4.0,5.0,998.0,57.0,50800.0,2.0,15.0,2.0,10.0,24763.0,13761.0,8.3,0.0,0.0,163.0,378.0,4.0,3.0,3.0,4.0,NaN,4.0,NaN,0.0,2.0,4.0,4.0,9.0,27.0,8.0,14.0,4.0,13.0,0.0,0.0,0.0,6.0,95.0,0.0,1.0,0.0,372872.0,99468.0,15000.0,94072.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,5000,5000,5000.0,36 months,17.97,180.69,D,D1,Administrative,6 years,MORTGAGE,59280.0,Source Verified,Dec-2018,Current,n,NaN,NaN,debt_consolidation,Debt cons

In [23]:
# Let's convert the issue_d column to actual date values that we can work with and create two new columns
loans['issue_year'] = pd.to_datetime(loans['issue_d']).dt.year
loans['issue_month'] = pd.to_datetime(loans['issue_d']).dt.month

In [24]:
####### Remove older records #######

# Remove records prior to 2013
loans = loans[loans.issue_year >= 2013]

####### Simplify loan status #######

# Keep only records if loans status is Fully Paid or Charged Off
loans = loans.loc[loans.loan_status.isin(['Fully Paid', 'Charged Off'])]

####### Remove joint loan applications #######

# Keep only records if application_type is Individual
loans = loans.loc[loans.application_type.isin(['Individual'])]

In [25]:
####### Reduce number of features #######

key_features = [
    # Loan
    'loan_amnt', 'term', 'int_rate', 'installment', 'grade', 'issue_year',
    'purpose', 'out_prncp', 'total_rec_int', 'total_pymnt',
    
    # Applicant
    'emp_length', 'home_ownership', 'annual_inc',
    
    # Basic credit history
    'open_acc', 'total_acc', 'open_rv_24m', 'inq_fi', 'acc_open_past_24mths', 'avg_cur_bal', 
    'delinq_amnt', 'mort_acc', 'num_bc_sats', 'num_bc_tl', 'num_sats', 
    
    # Bad credit history
    'collections_12_mths_ex_med', 'pub_rec', 'delinq_2yrs', 'acc_now_delinq', 'mths_since_last_major_derog', 
    'mths_since_last_delinq', 'tot_coll_amt', 'mths_since_recent_bc_dlq', 'num_tl_120dpd_2m', 'num_tl_30dpd', 
    'num_tl_90g_dpd_24m', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'tax_liens', 
    
    # Fees
    'total_rec_late_fee',
    
    # Target
    'loan_status'
]

# Keep only key features
loans = loans[key_features]

loans.head()

,loan_amnt,term,int_rate,installment,grade,issue_year,purpose,out_prncp,total_rec_int,total_pymnt,emp_length,home_ownership,annual_inc,open_acc,total_acc,open_rv_24m,inq_fi,acc_open_past_24mths,avg_cur_bal,delinq_amnt,mort_acc,num_bc_sats,num_bc_tl,num_sats,collections_12_mths_ex_med,pub_rec,delinq_2yrs,acc_now_delinq,mths_since_last_major_derog,mths_since_last_delinq,tot_coll_amt,mths_since_recent_bc_dlq,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,pct_tl_nvr_dlq,percent_bc_gt_75,tax_liens,total_rec_late_fee,loan_status
186,4500,36 months,11.31,147.99,B,2018,credit_card,0.0,49.22,4549.217149,10+ years,RENT,38500.0,12.0,25.0,7.0,0.0,8.0,2428.0,0.0,0.0,5.0,6.0,12.0,0.0,0.0,0.0,0.0,25.0,25.0,306.0,NaN,0.0,0.0,0.0,91.7,0.0,0.0,0.0,Fully Paid
269,20000,60 months,17.97,507.55,D,2018,debt_consolidation,0.0,13.58,20013.577333,4 years,RENT,57000.0,10.0,16.0,3.0,0.0,3.0,3336.0,0.0,1.0,6.0,9.0,10.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,100.0,33.3,0.0,0.0,Fully Paid
271,6600,36 months,11.31,217.05,B,2018,credit_card,0.0,22.81,6622.809000,10+ years,RENT,45000.0,6.0,13.0,2.0,0.0,2.0,4473.0,0.0,0.0,4.0,4.0,6.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,84.6,50.0,0.0,0.0,Fully Paid
296,2500,36 months,13.56,84.92,C,2018,other,0.0,1.29,2501.285667,5 years,RENT,42000.0,3.0,6.0,2.0,2.0,3.0,6216.0,0.0,0.0,2.0,2.0,3.0,0.0,0.0,0.0,0.0,NaN,53.0,0.0,NaN,0.0,0.0,0.0,83.3,0.0,0.0,0.0,Fully Paid
369,4000,36 months,17.97,144.55,D,2018,house,0.0,49.92,4049.916667,5 years,MORTGAGE,60000.0,11.0,18.0,6.0,3.0,10.0,9687.0,0.0,2.0,5.0,5.0,11.0,0.0,0.0,0.0,0.0,38.0,38.0,0.0,NaN,0.0,0.0,0.0,88.2,0.0,0.0,0.0,Fully Paid


In [26]:
loans.loan_status.value_counts()

Fully Paid     945605
Charged Off    241612
Name: loan_status, dtype: int64

### 2.) Eliminate class imbalance

After examining the number of records remaining, we can see that the data set has imbalanced classes where only 20% of the records show a bad loan. Since we still have 241,612 bad loans to work with, we will down-sample our good loans to match our number of bad loans.

For future iterations of this project, we may consider using tree based algorithms which are more capable of handling imbalanced data sets.

In [27]:
# Import resampling module for sklearn
from sklearn.utils import resample

# Separate good vs bad loans
good_loans = loans[loans.loan_status == 'Fully Paid']
bad_loans = loans[loans.loan_status == 'Charged Off']
 
# Downsample good loans
good_loans_downsampled = resample(good_loans, replace=False, n_samples=241612, random_state=123)

# Recombine loans
loans = pd.concat([good_loans_downsampled, bad_loans])

In [28]:
# Confirm that are classes are now balanced
loans["loan_status"].value_counts()

Fully Paid     241612
Charged Off    241612
Name: loan_status, dtype: int64

### 3.) Deal with categorical and numeric features

### 3.1.) Deal with categorical features

There are many categorical features which will need to be encoded to numeric values before we can build our predictive model.

In [29]:
# Taking a look at the categorical features

loan_objects = loans.select_dtypes(include=['object']).copy()
loan_objects.head()

,term,grade,purpose,emp_length,home_ownership,loan_status
551023,36 months,C,debt_consolidation,10+ years,MORTGAGE,Fully Paid
1122212,60 months,C,debt_consolidation,2 years,RENT,Fully Paid
1573214,36 months,C,other,1 year,RENT,Fully Paid
703942,36 months,B,debt_consolidation,10+ years,OWN,Fully Paid
541532,36 months,B,debt_consolidation,4 years,OWN,Fully Paid


In [30]:
loan_objects[loan_objects.isnull().any(axis=1)]

,term,grade,purpose,emp_length,home_ownership,loan_status
575497,36 months,D,home_improvement,NaN,RENT,Fully Paid
1163943,36 months,D,debt_consolidation,NaN,RENT,Fully Paid
2049992,36 months,C,debt_consolidation,NaN,MORTGAGE,Fully Paid
2005570,60 months,B,debt_consolidation,NaN,OWN,Fully Paid
786125,36 months,D,debt_consolidation,NaN,RENT,Fully Paid
...,...,...,...,...,...,...
2260059,36 months,B,debt_consolidation,NaN,RENT,Charged Off
2260134,36 months,C,debt_consolidation,NaN,OWN,Charged Off
2260245,36 months,C,debt_consolidation,NaN,MORTGAGE,Charged Off
2260331,36 months,C,debt_consolidation,NaN,RENT,Charged Off


We can not only see that employment length has some empty values, but also that this feature could be converted to numeric instead of categorical values.

In [31]:
# Convert employment length feature to numeric values

def convert_employment_length(x):
    if x == '< 1 year':
        return 0.5
    if x == '1 year':
        return 1
    if x == '2 years':
        return 2
    if x == '3 years':
        return 3
    if x == '4 years':
        return 4
    if x == '5 years':
        return 5
    if x == '6 years':
        return 6
    if x == '7 years':
        return 7
    if x == '8 years':
        return 8
    if x == '9 years':
        return 9
    if x == '10+ years':
        return 10
    else:
        return 0

loans.emp_length = loans.emp_length.apply(convert_employment_length)

In [32]:
# Perform one hot encoding on remaining categorical values
loans = pd.get_dummies(loans, columns=['term', 'grade', 'purpose', 'home_ownership', 'loan_status'])

In [33]:
#loans.drop(labels='loan_status_Charged Off', axis=1)
loans = loans.drop(labels='loan_status_Charged Off', axis=1)

In [34]:
loans = loans.rename(columns={'loan_status_Fully Paid': 'loan_status'})

### 3.2.) Deal with numeric features

*Replace NaN values* - Numeric features such as mths_since_last_delinq will include a NaN value if the loan applicant does not meet that given criteria. We will want to convert these values to 0.

In [35]:
# Let's take a look at features with NaN values

loan_nums = loans.select_dtypes(exclude=['object']).copy()
loan_nums.isnull().sum().sort_values(ascending=False)

mths_since_recent_bc_dlq       360368
mths_since_last_major_derog    344167
inq_fi                         281140
open_rv_24m                    281140
mths_since_last_delinq         236638
                                ...  
grade_A                             0
term_ 60 months                     0
term_ 36 months                     0
total_rec_late_fee                  0
loan_amnt                           0
Length: 62, dtype: int64

In [36]:
# We will have to deal with the NaN values differently depending on the type of data point. For example,
# for mths_since_recent_bc_dlq (months since last bancard delinquency), lower values here are actually worse than
# higher values (representing more recent delinquencies). We would not want to convert NaN values to 0 because that
# would penalize loan applicants who have never had a delinquency. Instead we will convert these values to a
# high value so as to properly 'reward' users who have never had a delinquency.

# In other cases, we will simply want to convert NaN values to 0 such as in the case of inq_fi (number of personal
# finance inquiries) wherein a value of 0 would be appropriate if a user has made no inquiries.

# Each feature will have to be reviewed manually to determine an appropriate value to be replaced. In the case of 
# mths_since_recent_bc_dlq, the highest value is 202 months. For users who have never had a delinquency it seems
# appropriate to replace NaN with a value of 1000.

# 'mths_since_recent_bc_dlq'   <-- Convert NaN to 1000
# 'mths_since_last_major_derog'   <-- Convert NaN to 1000
# 'inq_fi'   <-- Convert NaN to 0
# 'open_rv_24m'   <-- Convert NaN to 0
# 'mths_since_last_delinq'   <-- Convert NaN to 1000
# 'num_tl_120dpd_2m'   <-- Convert NaN to 0
# 'percent_bc_gt_75'   <-- Convert NaN to 0
# 'pct_tl_nvr_dlq'   <-- Convert NaN to 100
# 'avg_cur_bal'   <-- Convert NaN to 0

loans = loans.fillna(value={
'mths_since_recent_bc_dlq': 1000,
'mths_since_last_major_derog': 1000,
'inq_fi': 0,
'open_rv_24m': 0,
'mths_since_last_delinq': 1000,
'num_tl_120dpd_2m': 0,
'percent_bc_gt_75': 0,
'pct_tl_nvr_dlq': 100,
'avg_cur_bal': 0
})

## Predictive Modeling

Now that we have performed feature engineering, we are ready to test a few models.

#### Separate feature matrix and response

In [37]:
# Create feature matrix
features = ['delinq_amnt', 'num_tl_120dpd_2m', 'mths_since_recent_bc_dlq', 'tot_coll_amt', 'mths_since_last_delinq', 'mths_since_last_major_derog', 'acc_now_delinq', 'delinq_2yrs', 'pub_rec', 'collections_12_mths_ex_med', 'num_sats', 'num_bc_tl', 'num_bc_sats', 'mort_acc', 'avg_cur_bal', 'home_ownership_RENT', 'acc_open_past_24mths', 'inq_fi', 'open_rv_24m', 'total_acc', 'open_acc', 'annual_inc', 'emp_length', 'total_pymnt', 'total_rec_int', 'out_prncp', 'issue_year', 'installment', 'int_rate', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'home_ownership_OWN', 'home_ownership_MORTGAGE', 'home_ownership_ANY', 'purpose_wedding', 'purpose_vacation', 'purpose_small_business', 'purpose_renewable_energy', 'purpose_other', 'purpose_moving', 'purpose_medical', 'purpose_major_purchase', 'purpose_house', 'purpose_home_improvement', 'purpose_debt_consolidation', 'purpose_credit_card', 'purpose_car', 'grade_G', 'grade_F', 'grade_E', 'grade_D', 'grade_C', 'grade_B', 'grade_A', 'term_ 60 months', 'term_ 36 months', 'total_rec_late_fee', 'tax_liens', 'loan_amnt']

X = loans[features]
y = loans.loan_status

#### Train/Test Split and Standardize our Data

StandardScaler standardizes our data by subtracting the mean from each feature and dividing by its standard deviation. We will apply this to our features.

In [38]:
# Split data into training and test groups
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=100)

In [39]:
# Standardize the data using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Apply KNN Model (K=5)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

y_pred_class = knn.predict(X_test)
print((metrics.accuracy_score(y_test, y_pred_class)))

## Which features determine a bad loan

Here we will want to determine which features are most influential in causing a loan to be 'bad'.